In [ ]:
%pip install numpy torch torchvision torchaudio scikit-image matplotlib

In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from skimage import io
from skimage.transform import resize
import torch.utils.data as data


In [3]:
data_directory = 'C:\Programming\python\Tuberculosis_detection_using_deep_learning_and_CNN\Tuberculosis_dataset\ChinaSet_AllFiles\ChinaSet_AllFiles\CXR_png'
# print(data_directory)


<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\honey\AppData\Local\Temp\ipykernel_21948\3694138203.py:1: SyntaxWarning: invalid escape sequence '\P'
  data_directory = 'C:\Programming\python\Tuberculosis_detection_using_deep_learning_and_CNN\Tuberculosis_dataset\ChinaSet_AllFiles\ChinaSet_AllFiles\CXR_png'


In [4]:
# Define how we want to prepare our images for the neural network
# This converts the image to a format the network can understand
data_transforms = transforms.Compose([transforms.ToTensor()])

# This function loads and prepares each image for our neural network
def CNNloader(data_root, filename):
    # Load the image and resize it to 224x224 pixels
    filename_actual = data_root + '/' + filename
    data_old = io.imread(filename_actual)
    data_old = resize(data_old, (224, 224))
    data_old = np.array(data_old, dtype=np.float32)
    
    # If the image is black and white, convert it to a color image
    # (Our neural network expects color images)
    if len(data_old.shape) <= 2:
        data_new = np.zeros(data_old.shape + (3,))
        data_new[:,:,0] = data_new[:,:,1] = data_new[:,:,2] = np.array(data_old)
        data_old = np.array(data_new, dtype=np.float32)
    
    # Apply our predefined transformations
    data_old = data_transforms(np.array(data_old))
    return data_old

# This class helps organize our data and labels
class CNNDataLayer(data.Dataset):
    def __init__(self, data_root, filenames, loader):
        self.data_root = data_root
        self.filenames = filenames
        self.loader = loader

    # This tells the computer how to get each image and its label
    def __getitem__(self, index):
        filename = self.filenames[index]
        # We assume the label (0 or 1) is the second-to-last character in the filename
        target = [int(filename[-5])]
        target = torch.from_numpy(np.array(target))
        data = self.loader(self.data_root, filename)
        return data, target

    # This tells the computer how many images we have
    def __len__(self):
        return len(self.filenames)

# Check if we can use a GPU (which is faster) or if we need to use a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Get a list of all our image files and shuffle them randomly
all_files = np.array(os.listdir(data_directory))
np.random.shuffle(all_files)

# Split our data: 70% for training, 30% for testing
train_files = all_files[:round(0.7*all_files.shape[0])]
test_files = all_files[round(0.7*all_files.shape[0]):]

# Create our datasets using the custom class we defined earlier
data_sets_train = CNNDataLayer(data_root=data_directory, filenames=train_files, loader=CNNloader)
data_sets_test = CNNDataLayer(data_root=data_directory, filenames=test_files, loader=CNNloader)

# Create data loaders that will help feed data to our neural network in batches
data_loaders_train = data.DataLoader(data_sets_train, batch_size=8, shuffle=True, num_workers=0)
data_loaders_test = data.DataLoader(data_sets_test, batch_size=8, shuffle=True, num_workers=0)

# Load a pre-trained neural network (VGG16) that's good at understanding images
model_to_train = models.vgg16(pretrained=True)
# model_to_train.load_state_dict(torch.load("./vgg16-397923af.pth"))

# We don't want to retrain the whole network, so we "freeze" most of it
for param in model_to_train.features.parameters():
    param.require_grad = False

# Modify the last part of the network to focus on our specific task (finding tuberculosis)
num_features = model_to_train.classifier[6].in_features
features = list(model_to_train.classifier.children())[:-1]
features.extend([nn.Linear(num_features, 1), nn.Sigmoid()])
model_to_train.classifier = nn.Sequential(*features)

# Alternatively, we could use a custom-made neural network
# from own_cnn_model import SimpleCNN
# model_to_train = SimpleCNN()
model_to_train = model_to_train.to(device)

# Set up the "loss function" (how the network measures its mistakes)
# and the "optimizer" (how the network learns from its mistakes)
criterion = nn.BCELoss()
optimizer_ft = optim.SGD(model_to_train.parameters(), lr=0.001, momentum=0.9)

# Start the training process
num_epochs = 100  # We'll train for 100 rounds
for epoch in range(1, num_epochs+1):
    print('Epoch is ' + str(epoch))
    train_loss = 0
    test_loss = 0
    total_misclassified_train = total_count_train = 0
    total_misclassified_test = total_count_test = 0
    
    # Training phase
    model_to_train.train()
    for batch_idx, (data_now, target_now) in enumerate(data_loaders_train):
        # Move data to GPU if available
        data_now, target_now = data_now.to(device), target_now.to(device)
        
        # Make predictions
        target_output_model = model_to_train(data_now)
        target_now = target_now.type(torch.FloatTensor).to(device)
        
        # Calculate loss (how wrong the predictions were)
        target_loss = criterion(target_output_model, target_now)
        
        # Calculate how many predictions were wrong
        misclassified_temp = target_output_model[target_now == 1]
        total_misclassified_train += (misclassified_temp < 0.5).sum().item()
        misclassified_temp = target_output_model[target_now == 0]
        total_misclassified_train += (misclassified_temp >= 0.5).sum().item()
        total_count_train += data_now.shape[0]
        
        # Learn from the mistakes (backpropagation)
        optimizer_ft.zero_grad()
        target_loss.backward()
        optimizer_ft.step()
    
    # Testing phase (similar to training, but we don't learn from these)
    model_to_train.eval()
    with torch.no_grad():
        for batch_idx, (data_now, target_now) in enumerate(data_loaders_test):
            data_now, target_now = data_now.to(device), target_now.to(device)
            target_output_model = model_to_train(data_now)
            
            # Calculate how many predictions were wrong
            misclassified_temp = target_output_model[target_now == 1]
            total_misclassified_test += (misclassified_temp < 0.5).sum().item()
            misclassified_temp = target_output_model[target_now == 0]
            total_misclassified_test += (misclassified_temp >= 0.5).sum().item()
            total_count_test += data_now.shape[0]
    
    # Save our progress (the network's current state)
    snapshot_path = './snapshots_trial'
    os.makedirs(snapshot_path, exist_ok=True)
    snapshot_name = f'epoch-{epoch}-trainerror-{total_misclassified_train/total_count_train:.4f}-testerror-{total_misclassified_test/total_count_test:.4f}.pth'
    torch.save(model_to_train.state_dict(), os.path.join(snapshot_path, snapshot_name))

# Example of how to load a saved model for later use
model_loaded = torch.load("./snapshots/epoch-5-trainerror-0.1447-testerror-0.1558.pth", map_location=lambda storage, loc: storage)
model_to_train.load_state_dict(model_loaded)

c:\Programming\python\Tuberculosis_detection_using_deep_learning_and_CNN\env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Programming\python\Tuberculosis_detection_using_deep_learning_and_CNN\env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch is 1


MemoryError: Unable to allocate 205. MiB for an array with shape (2989, 2992, 3) and data type float64